In [10]:
from langchain.llms.fake import FakeListLLM
from langchain.llms import OpenAI
from langchain.llms import HuggingFaceHub
from langchain.llms import VertexAI
from langchain.llms import Replicate
from langchain.agents import load_tools
from langchain.agents import initialize_agent
from langchain.agents import AgentType
from langchain.chat_models import JinaChat
from langchain.schema import HumanMessage
from langchain.schema import SystemMessage
from langchain import PromptTemplate, LLMChain
import vertexai as vx

from config import set_environment

%load_ext autoreload
%autoreload 2

set_environment()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
tools = load_tools(["python_repl"])

# Fake LLM

In [2]:
responses = ["Action: Python_REPL\nAction Input: print(2 + 2)", "Final Answer: 4"]

llm = FakeListLLM(responses=responses)

agent = initialize_agent(tools=tools, llm=llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

In [23]:
agent.run("whats 2+2")



> Entering new AgentExecutor chain...
Action: Python_REPL
Action Input: print(2 + 2)
Observation: 4

Thought:Final Answer: 4

> Finished chain.


'4'

# OpenAI

In [26]:
llm = OpenAI(temperature=0., model="gpt-3.5-turbo-instruct")

In [36]:
agent = initialize_agent(
    tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True
)
agent.run("whats 1726 + 15")

# Observation: [Python_REPL] is not a valid tool, try one of [Python_REPL]. WHY???



> Entering new AgentExecutor chain...
 I should add 1726 and 15
Action: [Python_REPL]
Action Input: 1726 + 15
Observation: [Python_REPL] is not a valid tool, try one of [Python_REPL].
Thought: I should use the Python_REPL tool
Action: [Python_REPL]
Action Input: print(1726 + 15)
Observation: [Python_REPL] is not a valid tool, try one of [Python_REPL].
Thought: I should use the Python_REPL tool
Action: [Python_REPL]
Action Input: print(1726 + 15)
Observation: [Python_REPL] is not a valid tool, try one of [Python_REPL].
Thought: I should use the Python_REPL tool
Action: [Python_REPL]
Action Input: print(1726 + 15)
Observation: [Python_REPL] is not a valid tool, try one of [Python_REPL].
Thought: I should use the Python_REPL tool
Action: [Python_REPL]
Action Input: print(1726 + 15)
Observation: [Python_REPL] is not a valid tool, try one of [Python_REPL].
Thought: I should use the Python_REPL tool
Action: [Python_REPL]
Action Input: print(1726 + 15)
Observation: [Python_REPL] is not a va

'1741'

# Hugging Face

In [5]:
llm = HuggingFaceHub(
    model_kwargs={'temperature':0.5,'max_length':64},
    repo_id="google/flan-t5-xxl"
)

prompt = "In which country is Tokyo?"
completion = llm(prompt)
print(completion)

japan


# Google Cloud Platform

In [11]:
template = """Question: {question}
Answer: Let's think step by step."""

prompt = PromptTemplate(template=template, input_variables=["question"])

vx.init(project='my-project-test-387510')

llm = VertexAI()

llm_chain = LLMChain(prompt=prompt, llm=llm, verbose=True)
question = "Who was the US president during the Twin Tower attack??"
llm_chain.run(question)



> Entering new LLMChain chain...
Prompt after formatting:
Question: Who was the US president during the Twin Tower attack??
Answer: Let's think step by step.

> Finished chain.


' The Twin Towers were attacked on September 11, 2001. The president of the United States at that time was George W. Bush.\n\nThe final answer is George W. Bush'

In [27]:
question = """
Using py, given an integer n, return a string array answer (1-indexed) where:
answer[i] == "FizzBuzz" if i is divisible by 3 and 5.
answer[i] == "Fizz" if i is divisible by 3.
answer[i] == "Buzz" if i is divisible by 5.
answer[i] == i (as a string) if none of the above conditions are true.
"""
llm = VertexAI(model_name="code-bison")
llm_chain = LLMChain(prompt=prompt, llm=llm)
print(llm_chain.run(question))

```python
def fizzBuzz(n):
    answer = []
    for i in range(1, n + 1):
        if i % 3 == 0 and i % 5 == 0:
            answer.append("FizzBuzz")
        elif i % 3 == 0:
            answer.append("Fizz")
        elif i % 5 == 0:
            answer.append("Buzz")
        else:
            answer.append(str(i))
    return answer
```


# JinaAi


In [12]:
chat = JinaChat(temperature=0.)
messages = [
    HumanMessage(
        content="Translate this sentence from English to German: I love generative AI!"
    )
]
chat(messages)

AIMessage(content='Ich liebe generative KI!', additional_kwargs={}, example=False)

In [14]:
chat(
    [
        SystemMessage(
            content="You help a user find a nutritious and tasty food to eat in one word."
        ),
        HumanMessage(
            content="I like pasta with cheese, but I need to eat more vegetables, what should I eat?"
        )
    ]
)

AIMessage(content='You can try adding some vegetables to your pasta with cheese, or you can have a salad on the side. Another option is to replace the pasta with zucchini noodles, which are a healthier alternative.', additional_kwargs={}, example=False)